# ASPDNet experiments
This file contains experiments for my adaptation of ASPDNet. To access GPUs/TPUs, I ran this file in Google Colaboratory.

## Setup
To pull from the GitHub repository in Colab:
```
%cd drive/MyDrive/Conservation\ Research/Code/counting-cranes
!git add .
!git stash
!git pull
```
----

In [1]:
#Mounting Google Drive...
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Will have to restart runtime after running this cell!
!pip install -r "/content/drive/MyDrive/Conservation Research/Code/counting-cranes/requirements.txt"

     |████████████████████████████████| 2.2MB 8.4MB/s 
     |████████████████████████████████| 17.4MB 158kB/s 
     |████████████████████████████████| 102kB 13.9MB/s 
     |████████████████████████████████| 15.3MB 206kB/s 
     |████████████████████████████████| 11.6MB 50.2MB/s 
     |████████████████████████████████| 808kB 36.8MB/s 
     |████████████████████████████████| 8.3MB 40.9MB/s 
     |████████████████████████████████| 27.4MB 97kB/s 
     |████████████████████████████████| 51.0MB 58kB/s 
     |████████████████████████████████| 38.2MB 81kB/s 
     |████████████████████████████████| 829kB 38.5MB/s 
     |████████████████████████████████| 10.6MB 35.9MB/s 
     |████████████████████████████████| 122kB 54.4MB/s 
     |████████████████████████████████| 276kB 59.6MB/s 
     |████████████████████████████████| 59.9MB 1.2MB/s 
     |████████████████████████████████| 317kB 56.6MB/s 
     |████████████████████████████████| 61kB 10.8MB/s 
     |████████████████████████████████| 1.3MB 40.4M

In [19]:
import json

config = json.load(open('/content/drive/MyDrive/Conservation Research/Code/counting-cranes/config.json', 'r'))
DATA_FP = config['data_filepath_colab']
CODE_FP = config['code_filepath_colab']
MODEL_SAVE_FP = config['model_saves_filepath_colab']
SEED = config['random_seed']
HYPERPARAMETERS = config['ASPDNet_params']

In [20]:
import sys
import os
sys.path.append(CODE_FP) 
sys.path.append(os.path.join(CODE_FP, 'density_estimation'))
sys.path.append(os.path.join(CODE_FP, 'object_detection')) #TODO: hacky solution... change this is in the original .py!
sys.path.append(os.path.join(CODE_FP, 'density_estimation', 'ASPDNet'))

from ASPDNet_model import *
from bird_dataset import *
from ASPDNet.model import ASPDNet

import torch
from torch.utils.data import DataLoader
from pytorch_lightning import seed_everything, Trainer
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import LearningRateMonitor

## Model training
If the Colab runtime disconnects, you'll have to load from a post-epoch `PyTorch Lightning` model save.

In [8]:
#Setting our random seed for all operations (PyTorch, numpy, python.random)
seed_everything(SEED);

Global seed set to 1693


In [7]:
#Getting the tile size to use
tile_size = tuple(config['tile_size'])
HYPERPARAMETERS

{'learning_rate': 1e-05}

In [9]:
#Creating a dataset and subsetting into train/validation/test splits - run the "seed_everything" directly before this cell!
bird_dataset_train = BirdDataset(root_dir = DATA_FP, 
                                 transforms = get_transforms(train = True),
                                 tiling_method = 'random', #random tiling for training
                                 annotation_mode = 'points', 
                                 num_tiles = 5, 
                                 max_neg_examples = 1, 
                                 tile_size = tile_size)
bird_dataset_eval = BirdDataset(root_dir = DATA_FP, 
                                transforms = get_transforms(train = False), 
                                tiling_method = 'w_o_overlap', #tiling w/o overlap for validation/testing
                                annotation_mode = 'points', 
                                tile_size = tile_size) 

#  here, we limit which PARENT images each set gets!
indices = torch.randperm(len(bird_dataset_train)).tolist()
dataset_train = torch.utils.data.Subset(bird_dataset_train, indices[ : 24]) #24 images in train
dataset_val = torch.utils.data.Subset(bird_dataset_eval, indices[24 : 28]) #4 images in val
dataset_test = torch.utils.data.Subset(bird_dataset_eval, indices[28 : ]) #6 images in test

In [13]:
#Wrapping our train/validation/test sets in DataLoaders
dataloader_train = DataLoader(dataset_train, batch_size = 1, shuffle = True, collate_fn = collate_tiles_density) #TODO: add batch size to ASPDNet hyperparams 
dataloader_val = DataLoader(dataset_val, batch_size = 1, shuffle = False, collate_fn = collate_tiles_density) 
dataloader_test = DataLoader(dataset_test, batch_size = 1, shuffle = False, collate_fn = collate_tiles_density) 

In [29]:
#Instantiating the pre-trained model and wrapping it in PyTorch Lightning class
model = ASPDNet()
pl_model = ASPDNetLightning(model = model, lr = HYPERPARAMETERS['learning_rate'])
pl_model; #checking out the model summary

In [31]:
#Setting up logger 
logging_directory = os.path.join(MODEL_SAVE_FP, 'ASPDNet', 'evaluation')
logger = CSVLogger(logging_directory, name = 'initial_training_6.17.2021') #REMEMBER: change this when you shift experiments!

In [32]:
#Training the model
checkpoint_dir = os.path.join(MODEL_SAVE_FP, 'ASPDNet', 'trainer_checkpoints')

#TODO: use this early stopping callback for ASPDNet or stick w/the paper's training workflow (just 800 epochs)??
# early_stopping_callback = EarlyStopping(monitor = 'Val_MAE', patience = 7, mode = 'min', min_delta = 5)
lr_callback = LearningRateMonitor(logging_interval = 'step')

trainer = Trainer(gpus = 1, max_epochs = 60, callbacks = [lr_callback], default_root_dir = checkpoint_dir, logger = logger) 
trainer.fit(pl_model, train_dataloader = dataloader_train)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | ASPDNet | 27.4 M
----------------------------------
27.4 M    Trainable params
0         Non-trainable params
27.4 M    Total params
109.687   Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


In [33]:
#Saving the model
save_name = 'ASPDNet_first_try_6.17.2021.pth'
torch.save(model.state_dict(), os.path.join(MODEL_SAVE_FP, 'ASPDNet', save_name))

## Model testing

In [36]:
#Loading the model
save_name = 'ASPDNet_first_try_6.17.2021.pth'
model = ASPDNet()
model.load_state_dict(torch.load(os.path.join(MODEL_SAVE_FP, 'ASPDNet', save_name)))
pl_model = ASPDNetLightning(model = model, lr = HYPERPARAMETERS['learning_rate'])

In [35]:
#Using the saved model and the test set to evaluate (via AP)
trainer = Trainer(gpus = 1)
results = trainer.test(pl_model, test_dataloaders = dataloader_test)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


TypeError: ignored